In [1]:
import os
import sys
import math
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import xgboost as xgb
import pickle
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, precision_score, recall_score, accuracy_score, f1_score
from sklearn.metrics import make_scorer, precision_recall_curve, auc
from sklearn.preprocessing import MinMaxScaler

from bayes_opt import BayesianOptimization
from collections import Counter
from imblearn.over_sampling import SMOTE

from scipy.stats import ks_2samp
def ks_stat(y, yhat):
    return ks_2samp(yhat[y==1], yhat[y!=1]).statistic

C:\Users\patrick.franco\AppData\Roaming\Python\Python311\site-packages\xgboost\compat.py:93: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


# Menu

<a name="navegacao"></a>

## 1) [Preparação dos dados](#parte1)
- 1.1 [Leitura base principal](#principal)
- 1.2 [Leitura base mes](#mes)
- 1.3 [Leitura base hora](#hora)
- 1.4 [Merge principal e base mensal](#merge1)
- 1.5 [Merge principal e base hora](#merge2)
- 1.6 [Confere marcação](#marcacao)


## 2 [Salvando as bases de treino](#parte2)
- 2.1 [Salvando base com histórico](#comhist)
- 2.2 [Salvando base sem histórico](#semhist)


<a name="principal"></a>

## 1.1) Leitura base principal


## Leitura dos dados

In [2]:
%%time
df = pd.read_csv("BNDES_UNIFICADO.csv",converters={'CNPJ8': str,'INTERMEDIARIA': str},
                 delimiter=";" , encoding='latin-1')

CPU times: total: 391 ms
Wall time: 362 ms


In [3]:
df.replace([np.inf, -np.inf], np.nan, inplace=True)
df.dropna(inplace=True)

In [4]:
df.shape

(125516, 27)

In [5]:
df.columns

Index(['CNPJ8', 'ANO', 'EMPRESA_PUBLICA', 'GARANTIA', 'INDIRETA', 'INOVACAO',
       'INSTRUMENTO', 'UF', 'INTERMEDIARIA', 'CUSTO', 'MEDIA_JUROS',
       'PRAZO_AMORTIZACAO', 'TESOURO', 'PRAZO_CARENCIA', 'VALOR_CONTRATO',
       'VALOR_DESENBOLSO', 'Porte_Cliente', 'CAPITAL_SOCIAL', 'IDADE',
       'NATJUR', 'NCONTRATOS', 'NFILIAIS', 'PORTE_RECEITA', 'SITUACAO',
       'IDADE_SOCIOS', 'QTDSOCIOS', 'SOCIO_PJ'],
      dtype='object')

In [6]:
%%time
df.head()

CPU times: total: 0 ns
Wall time: 0 ns


,CNPJ8,ANO,EMPRESA_PUBLICA,GARANTIA,INDIRETA,INOVACAO,INSTRUMENTO,UF,INTERMEDIARIA,CUSTO,...,CAPITAL_SOCIAL,IDADE,NATJUR,NCONTRATOS,NFILIAIS,PORTE_RECEITA,SITUACAO,IDADE_SOCIOS,QTDSOCIOS,SOCIO_PJ
0,00000000,2002,0,OUTROS,0,1,OUTROS,IE,00000000,TJLP,...,9.000002e+10,37.0,2.0,15,5089,5.0,1,1.0,1,0
1,00000000,2003,0,OUTROS,0,0,OUTROS,IE,92816560,TJLP,...,9.000002e+10,38.0,2.0,2,5123,5.0,1,1.0,1,0
2,00000000,2009,1,SEM GARANTIA,0,0,OUTROS,RJ,,TAXAFIXA,...,9.000002e+10,44.0,2.0,5,6912,5.0,1,1.0,1,0
3,00000000,2010,1,SEM GARANTIA,0,0,OUTROS,RJ,,TAXAFIXA,...,9.000002e+10,45.0,2.0,2,7002,5.0,1,1.0,1,0
4,00000000,2012,1,MISTA,0,0,OUTROS,IE,,TJLP,...,9.000002e+10,47.0,2.0,1,7407,5.0,1,1.0,1,0


In [7]:
%%time
df.dtypes

CPU times: total: 0 ns
Wall time: 0 ns


CNPJ8                 object
ANO                    int64
EMPRESA_PUBLICA        int64
GARANTIA              object
INDIRETA               int64
INOVACAO               int64
INSTRUMENTO           object
UF                    object
INTERMEDIARIA         object
CUSTO                 object
MEDIA_JUROS          float64
PRAZO_AMORTIZACAO    float64
TESOURO                int64
PRAZO_CARENCIA       float64
VALOR_CONTRATO       float64
VALOR_DESENBOLSO     float64
Porte_Cliente         object
CAPITAL_SOCIAL       float64
IDADE                float64
NATJUR               float64
NCONTRATOS             int64
NFILIAIS               int64
PORTE_RECEITA        float64
SITUACAO               int64
IDADE_SOCIOS         float64
QTDSOCIOS              int64
SOCIO_PJ               int64
dtype: object

In [8]:
pd.crosstab(df.PORTE_RECEITA,df.SITUACAO)

SITUACAO,0,1
PORTE_RECEITA,,
1.0,72792,2639
3.0,22618,1864
5.0,21244,4359


In [9]:
pd.crosstab(df.Porte_Cliente,df.SITUACAO)

SITUACAO,0,1
Porte_Cliente,,
GRANDE,3692,1219
MICRO,40280,1110
MÉDIA,20556,3430
PEQUENA,52126,3103


In [10]:
%%time
pd.crosstab(df.CUSTO, df.SITUACAO )

CPU times: total: 31.2 ms
Wall time: 26.8 ms


SITUACAO,0,1
CUSTO,,
CDI,14,2
IPCA,48,8
OUTROS,248,119
SELIC,28761,1823
TAXAFIXA,30327,1840
TJLP,20437,2296
TLP,36819,2774


In [11]:
%%time
pd.crosstab(df.NATJUR, df.SITUACAO)

CPU times: total: 15.6 ms
Wall time: 23.1 ms


SITUACAO,0,1
NATJUR,,
1.0,184,49
2.0,116254,8786
3.0,187,21
4.0,29,6


In [12]:
pd.crosstab(df.EMPRESA_PUBLICA, df.SITUACAO)

SITUACAO,0,1
EMPRESA_PUBLICA,,
0,116375,8770
1,279,92


In [13]:
pd.crosstab(df.INDIRETA, df.SITUACAO)

SITUACAO,0,1
INDIRETA,,
0,2711,792
1,113943,8070


In [14]:
pd.crosstab(df.INOVACAO, df.SITUACAO)

SITUACAO,0,1
INOVACAO,,
0,116310,8730
1,344,132


In [15]:
pd.crosstab(df.TESOURO, df.SITUACAO)

SITUACAO,0,1
TESOURO,,
0,110155,7886
1,6499,976


In [16]:
pd.crosstab(df.SOCIO_PJ, df.SITUACAO)

SITUACAO,0,1
SOCIO_PJ,,
0,116654,8862


In [17]:
%%time
df.loc[ (df['CAPITAL_SOCIAL']<1)     ,'CAPITAL_SOCIAL']    = 1
df.loc[ (df['IDADE']<1)              ,'IDADE']             = 1
df.loc[ (df['NCONTRATOS']<1)         ,'NCONTRATOS']        = 1
df.loc[ (df['NFILIAIS']<1)           ,'NFILIAIS']          = 1
df.loc[ (df['IDADE_SOCIOS']<1)       ,'IDADE_SOCIOS']      = 1
df.loc[ (df['QTDSOCIOS']<1)          ,'QTDSOCIOS']         = 1
df.loc[ (df['MEDIA_JUROS']<1)        ,'MEDIA_JUROS']       = 1
df.loc[ (df['PRAZO_AMORTIZACAO']<1)  ,'PRAZO_AMORTIZACAO'] = 1
df.loc[ (df['PRAZO_CARENCIA']<1)     ,'PRAZO_CARENCIA']    = 1
df.loc[ (df['VALOR_CONTRATO']<1)     ,'VALOR_CONTRATO']    = 1
df.loc[ (df['VALOR_DESENBOLSO']<1)   ,'VALOR_DESENBOLSO']  = 1

CPU times: total: 15.6 ms
Wall time: 8.98 ms


In [18]:
%%time
df['enc_NATJUR']        = df.NATJUR.astype("category").cat.codes
df['enc_GARANTIA']      = df.GARANTIA.astype("category").cat.codes
df['enc_INSTRUMENTO']   = df.INSTRUMENTO.astype("category").cat.codes
df['enc_CUSTO']         = df.CUSTO.astype("category").cat.codes
df['enc_PORTE_CLIENTE'] = df.Porte_Cliente.astype("category").cat.codes
df['enc_PORTE_RECEITA'] = df.PORTE_RECEITA.astype("category").cat.codes
df['enc_SITUACAO']      = df.SITUACAO.astype("category").cat.codes
df['enc_UF']            = df.UF.astype("category").cat.codes

CPU times: total: 62.5 ms
Wall time: 59.1 ms


In [19]:
%%time
df['ln_capsoc']        = np.log(df['CAPITAL_SOCIAL']+1)
df['ln_idade']         = np.log(df['IDADE']+1)
df['ln_contratos']     = np.log(df['NCONTRATOS']+1)
df['ln_filiais']       = np.log(df['NFILIAIS']+1)
df['ln_sociosage']     = np.log(df['IDADE_SOCIOS']+1)
df['ln_qtdsocios']     = np.log(df['QTDSOCIOS']+1)
df['ln_juros']         = np.log(df['MEDIA_JUROS']+1)
df['ln_amortizacao']   = np.log(df['PRAZO_AMORTIZACAO']+1)
df['ln_carencia']      = np.log(df['PRAZO_CARENCIA']+1)
df['ln_vlrcontrato']   = np.log(df['VALOR_CONTRATO']+1)
df['ln_vlrdesembolso'] = np.log(df['VALOR_DESENBOLSO']+1)

CPU times: total: 15.6 ms
Wall time: 25.3 ms


## Modelo sem histórico foi treinado com as variáveis na seguinte ordem:
['faixa_hora', 'vl_medio_mes_atual', 'dif_vl_1', 'tres_prim_dig_codbarras', 'pagador_pf', 'dif_vl_4', 'dia_do_mes', 'qtd_operacoes_mes_corrente', 'vl_medio_dia_corrente', 'sec_dig', 'qtd_operacoes_dia_corrente', 'qtd_trn_60min', 'centavos', 'dia_da_semana']

In [20]:
%%time
files = df.columns
#selected_files = files.str.contains('ln_|enc_|INDIRETA|EMPRESA_PUBLICA|INOVACAO|TESOURO|SOCIO_PJ')
selected_files = files.str.contains('ln_|INDIRETA|EMPRESA_PUBLICA|INOVACAO|TESOURO|SOCIO_PJ')
atributes = files[selected_files]

CPU times: total: 0 ns
Wall time: 0 ns


In [21]:
atributes

Index(['EMPRESA_PUBLICA', 'INDIRETA', 'INOVACAO', 'TESOURO', 'SOCIO_PJ',
       'ln_capsoc', 'ln_idade', 'ln_contratos', 'ln_filiais', 'ln_sociosage',
       'ln_qtdsocios', 'ln_juros', 'ln_amortizacao', 'ln_carencia',
       'ln_vlrcontrato', 'ln_vlrdesembolso'],
      dtype='object')

In [23]:
df[atributes].dtypes

EMPRESA_PUBLICA       int64
INDIRETA              int64
INOVACAO              int64
TESOURO               int64
SOCIO_PJ              int64
ln_capsoc           float64
ln_idade            float64
ln_contratos        float64
ln_filiais          float64
ln_sociosage        float64
ln_qtdsocios        float64
ln_juros            float64
ln_amortizacao      float64
ln_carencia         float64
ln_vlrcontrato      float64
ln_vlrdesembolso    float64
dtype: object

In [22]:
df[atributes].describe().transpose()

,count,mean,std,min,25%,50%,75%,max
EMPRESA_PUBLICA,125516.0,0.002956,0.054287,0.000000,0.000000,0.000000,0.000000,1.000000
INDIRETA,125516.0,0.972091,0.164712,0.000000,1.000000,1.000000,1.000000,1.000000
INOVACAO,125516.0,0.003792,0.061465,0.000000,0.000000,0.000000,0.000000,1.000000
TESOURO,125516.0,0.059554,0.236660,0.000000,0.000000,0.000000,0.000000,1.000000
SOCIO_PJ,125516.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
ln_capsoc,125516.0,10.674521,3.548741,0.693147,9.615872,11.002117,11.982935,26.048381
ln_idade,125516.0,1.250731,0.858364,0.693147,0.693147,0.693147,1.791759,4.718499
ln_contratos,125516.0,0.813766,0.313496,0.693147,0.693147,0.693147,0.693147,5.613128
ln_filiais,125516.0,0.876072,0.495155,0.693147,0.693147,0.693147,0.693147,8.921591
ln_sociosage,125516.0,2.572323,1.539442,0.693147,0.693147,3.610918,3.850148,4.510860


## SMOTE: Synthetic Minority Oversampling Technique

In [23]:
y0 = df['SITUACAO'].values.reshape(-1, 1)
X0 = df[atributes]
print(X0.shape,y0.shape)

(125516, 16) (125516, 1)


In [24]:
#import joblib
#scaler = joblib.load("scaler.saved") 

from numpy import asarray
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
# transform data
X0 = scaler.fit_transform(df[atributes])

In [25]:
# base sintética: treino e teste
x_train  , x_test0 , y_train, y_test0 = train_test_split(X0, y0, test_size = 0.4, random_state=123)

# base sintética: teste e out of sample
x_test , x_out , y_test, y_out = train_test_split(x_test0, y_test0, test_size = 0.4, random_state=123)

In [26]:
# summarize the new class distribution
#counter0 = Counter(y_train)
#counter1 = Counter(y_test)
#counter2 = Counter(y_out)
#print(counter0, counter1, counter2)

In [ ]:
%%time
from bayes_opt import BayesianOptimization
from sklearn.metrics import roc_auc_score

gr_range = ['depthwise','lossguide']
bs_range = ['gbtree','dart']

def gbm_xgb(learning_rate, max_depth, gamma, min_child_weight, subsample, eta, reg_alpha, reg_lambda,
            n_estimators, max_delta_step, max_leaves, grow_policy, scale_pos_weight, booster , max_bin):
    model = xgb.XGBClassifier(
                  learning_rate       = learning_rate,
                  max_depth           = int(max_depth),
                  gamma               = gamma,
                  min_child_weight    = int(min_child_weight),
                  subsample           = subsample,
                  eta                 = eta,
                  reg_alpha           = reg_alpha,
                  reg_lambda          = reg_lambda,
                  n_estimators        = int(n_estimators),
                  max_delta_step      = max_delta_step,
                  max_leaves          = int(max_leaves),
                  max_bin             = int(max_bin),
                  booster             = bs_range[int(booster)],
                  grow_policy         = gr_range[int(grow_policy)],
                  scale_pos_weight    = scale_pos_weight,
                  missing             = 0,
                  random_state        = 666,
                  nthread =10 )
    
    model.fit(x_train, y_train.ravel(), verbose=False)
    pred_labels = model.predict(x_test)
    return roc_auc_score(y_test, pred_labels)

params_xgb = {
    'learning_rate'          : (0.010 ,0.500),
    'max_depth'              : (2.000 ,11.00),
    'gamma'                  : (1.000 ,100.0),
    'min_child_weight'       : (1.000 ,100.0),
    'subsample'              : (0.222 ,0.999),
    'eta'                    : (0.005 ,0.500),
    'reg_alpha'              : (0.050 ,10.00),
    'reg_lambda'             : (0.050 ,10.00),
    'n_estimators'           : (50.00 ,500.0),
    'max_delta_step'         : (0.005 ,5.000),
    'max_leaves'             : (2.000 ,50.00),
    'max_bin'                : (2.000 ,100.0),
    'booster'                : (0.001 ,1.000),
    'grow_policy'            : (0.001 ,1.000),
    'scale_pos_weight'       : (0.222 ,100.0),
}
   
xgb0 = BayesianOptimization(f=gbm_xgb, pbounds=params_xgb, random_state=123, allow_duplicate_points=True) 
xgb0.set_gp_params(alpha=1e-4)
xgb0.maximize(init_points=30, n_iter=270)

In [34]:
#params_xgb = xgb0.max['params']
params_xgb = {'booster': 0.7112876949019146,
 'eta': 0.4856678395525138,
 'gamma': 87.29661038108841,
 'grow_policy': 0.7104514896693779,
 'learning_rate': 0.47966977408061673,
 'max_bin': 44.121707112885026,
 'max_delta_step': 4.365030177037427,
 'max_depth': 5.203619011762532,
 'max_leaves': 46.62865533938464,
 'min_child_weight': 15.728987967968257,
 'n_estimators': 473.0130567177231,
 'reg_alpha': 8.335526162967966,
 'reg_lambda': 8.468245640054818,
 'scale_pos_weight': 12.586790084191318,
 'subsample': 0.6854703200115296}

In [35]:
gr_range = ['depthwise','lossguide']
bs_range = ['gbtree','dart']

params_xgb['max_depth']        = int(params_xgb['max_depth'])
params_xgb['min_child_weight'] = int(params_xgb['min_child_weight'])
params_xgb['n_estimators']     = int(params_xgb['n_estimators'])
params_xgb['max_leaves']       = int(params_xgb['max_leaves'])
params_xgb['max_bin']          = int(params_xgb['max_bin'])
params_xgb['booster']          = bs_range[int(params_xgb['booster'])]
params_xgb['grow_policy']      = gr_range[int(params_xgb['grow_policy'])]

params_xgb

{'booster': 'gbtree',
 'eta': 0.4856678395525138,
 'gamma': 87.29661038108841,
 'grow_policy': 'depthwise',
 'learning_rate': 0.47966977408061673,
 'max_bin': 44,
 'max_delta_step': 4.365030177037427,
 'max_depth': 5,
 'max_leaves': 46,
 'min_child_weight': 15,
 'n_estimators': 473,
 'reg_alpha': 8.335526162967966,
 'reg_lambda': 8.468245640054818,
 'scale_pos_weight': 12.586790084191318,
 'subsample': 0.6854703200115296}

In [ ]:
learning_rate       = xgb0.max['params']['learning_rate']
max_depth           = int(xgb0.max['params']['max_depth'])
gamma               = xgb0.max['params']['gamma']
min_child_weight    = int(xgb0.max['params']['min_child_weight'])
subsample           = xgb0.max['params']['subsample']
eta                 = xgb0.max['params']['eta']
reg_alpha           = xgb0.max['params']['reg_alpha']
reg_lambda          = xgb0.max['params']['reg_lambda']
n_estimators        = int(xgb0.max['params']['n_estimators'])
max_delta_step      = xgb0.max['params']['max_delta_step']
max_leaves          = int(xgb0.max['params']['max_leaves'])
max_bin             = int(xgb0.max['params']['max_bin'])
booster             = bs_range[int(xgb0.max['params']['booster'])]
grow_policy         = gr_range[int(xgb0.max['params']['grow_policy'])]
scale_pos_weight    = xgb0.max['params']['scale_pos_weight']

print('\n learning_rate:'     , learning_rate,
      '\n max_depth:'         , max_depth,
      '\n gamma:'             , gamma,
      '\n min_child_weight:'  , min_child_weight,
      '\n subsample:'         , subsample,
      '\n eta:'               , eta,
      '\n reg_alpha:'         , reg_alpha,
      '\n reg_lambda:'        , reg_lambda,
      '\n n_estimators:'      , n_estimators,
      '\n max_delta_step:'    , max_delta_step,
      '\n max_leaves:'        , max_leaves,
      '\n max_bin:'           , max_bin,
      '\n grow_policy:'       , grow_policy,
      '\n booster:'           , booster,
      '\n scale_pos_weight:'  , scale_pos_weight)

In [ ]:
%%time
cbbb= xgb.XGBClassifier(learning_rate     = learning_rate,
                        max_depth         = max_depth,
                        gamma             = gamma,
                        min_child_weight  = min_child_weight,
                        subsample         = subsample,
                        eta               = eta,
                        reg_alpha         = reg_alpha,
                        reg_lambda        = reg_lambda,
                        n_estimators      = n_estimators,
                        max_delta_step    = max_delta_step,
                        max_leaves        = max_leaves,
                        max_bin           = max_bin,
                        grow_policy       = grow_policy,
                        booster           = booster,
                        scale_pos_weight  = scale_pos_weight)
cbbb.fit(x_train, y_train.ravel())

In [39]:
%%time
cbbb= xgb.XGBClassifier(**params_xgb)
cbbb.fit(x_train, y_train.ravel())

CPU times: total: 1min 44s
Wall time: 14.8 s


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, eta=0.4856678395525138,
              gamma=87.29661038108841, gpu_id=-1, grow_policy='depthwise',
              importance_type='gain', interaction_constraints='',
              learning_rate=0.47966977408061673, max_bin=44,
              max_delta_step=4.365030177037427, max_depth=5, max_leaves=46,
              min_child_weight=15, missing=nan, monotone_constraints='()',
              n_estimators=473, n_jobs=0, num_parallel_tree=1, random_state=0,
              reg_alpha=8.335526162967966, reg_lambda=8.468245640054818,
              scale_pos_weight=12.586790084191318, subsample=0.6854703200115296,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [40]:
%%time
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score, roc_auc_score
print("Accuracy:" , accuracy_score(  y_train, cbbb.predict(x_train))) 
print("F1 score:" , f1_score(        y_train, cbbb.predict(x_train))) 
print("Recall:"   , recall_score(    y_train, cbbb.predict(x_train))) 
print("Precision:", precision_score( y_train, cbbb.predict(x_train))) 
print("KS score:" , ks_stat(         y_train.ravel(), cbbb.predict_proba(x_train)[:,1] ))
print("AUC score:", roc_auc_score(   y_train, cbbb.predict_proba(x_train)[:,1]))

Accuracy: 0.8586888685283299
F1 score: 0.4926098979689139
Recall: 0.9643457158857569
Precision: 0.3307933662034962
KS score: 0.817645390717586
AUC score: 0.9408950451700975
CPU times: total: 3.23 s
Wall time: 685 ms


In [41]:
%%time
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score
print("Accuracy:" ,accuracy_score(  y_test, cbbb.predict(x_test))) 
print("F1 score:" ,f1_score(        y_test, cbbb.predict(x_test))) 
print("Recall:"   ,recall_score(    y_test, cbbb.predict(x_test))) 
print("Precision:",precision_score( y_test, cbbb.predict(x_test))) 
print("KS score:" , ks_stat(        y_test.ravel(), cbbb.predict_proba(x_test)[:,1] ))
print("AUC score:", roc_auc_score(  y_test, cbbb.predict_proba(x_test)[:,1]))

Accuracy: 0.856327180985261
F1 score: 0.4869606448553817
Recall: 0.9706994328922496
Precision: 0.325
KS score: 0.8185643286363227
AUC score: 0.9387236157614163
CPU times: total: 1.12 s
Wall time: 323 ms


In [42]:
%%time
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score
print("Accuracy:" ,accuracy_score(  y_out, cbbb.predict(x_out))) 
print("F1 score:" ,f1_score(        y_out, cbbb.predict(x_out))) 
print("Recall:"   ,recall_score(    y_out, cbbb.predict(x_out))) 
print("Precision:",precision_score( y_out, cbbb.predict(x_out))) 
print("KS score:" , ks_stat(        y_out.ravel(), cbbb.predict_proba(x_out)[:,1] ))
print("AUC score:", roc_auc_score(  y_out, cbbb.predict_proba(x_out)[:,1]))

Accuracy: 0.8586864512274063
F1 score: 0.4866136034732272
Recall: 0.9683225341972642
Precision: 0.3249577192558589
KS score: 0.8198992866277341
AUC score: 0.9390985107197629
CPU times: total: 859 ms
Wall time: 195 ms


## Leitura dos dados originais

In [43]:
%%time
df['proba'] = cbbb.predict_proba(X0)[:,1]

CPU times: total: 359 ms
Wall time: 94.8 ms


In [44]:
df[['SITUACAO','proba']].groupby(['SITUACAO']).count()

,proba
SITUACAO,
0,116654
1,8862


In [45]:
%%time
df[['SITUACAO','proba']].groupby(['SITUACAO']).describe()

CPU times: total: 31.2 ms
Wall time: 25 ms


proba                                                    \
             count      mean       std       min       25%       50%   
SITUACAO                                                               
0         116654.0  0.151084  0.292995  0.001301  0.007122  0.011834   
1           8862.0  0.844095  0.132819  0.004927  0.842715  0.878845   

                              
               75%       max  
SITUACAO                      
0         0.063389  0.953062  
1         0.903306  0.953062

In [46]:
%%time
df.loc[ df['proba']>=0.84 ,'PRED'] = 1
df.loc[ df['proba'] <0.84 ,'PRED'] = 0

CPU times: total: 0 ns
Wall time: 1.79 ms


<a name="metricas"></a>


# 3) Métricas
  
- ir para [Menu Principal](#navegacao)

<a name="amostra"></a>


## 3.1) Métricas na Amostra
  
- ir para [Menu Principal](#navegacao)

In [47]:
from sklearn.metrics import confusion_matrix
confusao_pop = confusion_matrix(df['SITUACAO'], df['PRED'])
print("Confusion matrix for test:\n%s" % confusao_pop )

Confusion matrix for test:
[[106178  10476]
 [  2152   6710]]


In [48]:
%%time
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score

print("Accuracy:" , accuracy_score(df['SITUACAO'], df['PRED'])) 
print("F1 score:" , f1_score(df['SITUACAO'], df['PRED'])) 
print("Recall:"   , recall_score(df['SITUACAO'], df['PRED'])) 
print("Precision:", precision_score(df['SITUACAO'], df['PRED'])) 

Accuracy: 0.899391312661334
F1 score: 0.5152027027027026
Recall: 0.7571654254118709
Precision: 0.3904340742464797
CPU times: total: 188 ms
Wall time: 190 ms


<a name="falsopos"></a>

## 3.2) Taxa de Falso Positivo
  
- ir para [Menu Principal](#navegacao)

In [49]:
%%time
tn, fp, fn, tp = confusao_pop.ravel()
print('FPR:', fp/(fp + tn))
print('TPR:', tp/(tp + fn))

FPR: 0.08980403586675124
TPR: 0.7571654254118709
CPU times: total: 0 ns
Wall time: 997 µs


In [50]:
metrics_list_safra = []
safras = df['ANO'].unique()
safras.sort()
i=0
for ano in safras:
    metrics = dict()
    metrics['Model Index']=i
    metrics['ANO']=ano
    valid_hue = df[df['ANO']==ano]
    print(ano)
    confusao_pop = confusion_matrix(valid_hue['SITUACAO'], valid_hue['PRED'])
    
    metrics["Accuracy"]  = accuracy_score(valid_hue['SITUACAO'] , valid_hue['PRED'])
    metrics['AUC']       = roc_auc_score(valid_hue['SITUACAO']  , valid_hue['proba'])
    metrics['KS Score']  = ks_stat(valid_hue['SITUACAO']        , valid_hue['proba'])
    metrics["F1 score"]  = f1_score(valid_hue['SITUACAO']       , valid_hue['PRED'])
    metrics["Recall"]    = recall_score(valid_hue['SITUACAO']   , valid_hue['PRED']) 
    metrics["Precision"] = precision_score(valid_hue['SITUACAO'], valid_hue['PRED'])
    metrics["False positive rate"] = confusao_pop[0][1]/(confusao_pop[0][0]+confusao_pop[0][1])
    metrics["True positive rate"]  = confusao_pop[1][1]/(confusao_pop[1][0]+confusao_pop[1][1])
    metrics_list_safra.append(metrics)
    del valid_hue, confusao_pop

2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2022


In [51]:
df_metrics = pd.DataFrame(metrics_list_safra)

In [52]:
df_metrics.to_csv('df_metrics_xbt.csv')

In [53]:
df_metrics

,Model Index,ANO,Accuracy,AUC,KS Score,F1 score,Recall,Precision,False positive rate,True positive rate
0,0,2002,0.881579,0.917424,0.739394,0.929134,0.893939,0.967213,0.200000,0.893939
1,0,2003,0.833333,0.895508,0.625000,0.882353,0.937500,0.833333,0.375000,0.937500
2,0,2004,0.815385,0.920000,0.709524,0.833333,0.857143,0.810811,0.233333,0.857143
3,0,2005,0.620370,0.721301,0.453514,0.506024,0.724138,0.388889,0.417722,0.724138
4,0,2006,0.741007,0.828965,0.699115,0.538462,0.807692,0.403846,0.274336,0.807692
5,0,2007,0.668639,0.822368,0.573726,0.500000,0.777778,0.368421,0.360902,0.777778
6,0,2008,0.725275,0.814725,0.621249,0.545455,0.810811,0.410959,0.296552,0.810811
7,0,2009,0.739464,0.873712,0.645749,0.558442,0.826923,0.421569,0.282297,0.826923
8,0,2010,0.692661,0.833547,0.524784,0.608187,0.881356,0.464286,0.377358,0.881356
9,0,2011,0.725490,0.827086,0.588235,0.539474,0.803922,0.405941,0.294118,0.803922


In [54]:

dataframe=pd.DataFrame(cbbb.feature_importances_, columns=['feature_importances'])
dataframe['features'] = list(atributes)
dataframe.sort_values('feature_importances',inplace=True,ascending=False)
dataframe

,feature_importances,features
8,0.849262,ln_filiais
6,0.037258,ln_idade
5,0.020132,ln_capsoc
11,0.016954,ln_juros
7,0.016457,ln_contratos
9,0.014505,ln_sociosage
14,0.012706,ln_vlrcontrato
12,0.011513,ln_amortizacao
13,0.009917,ln_carencia
15,0.005734,ln_vlrdesembolso


In [55]:
dataframe.to_csv('feature_importances_xbt.csv')

In [56]:
df.proba.quantile([0.01,0.025,0.05,0.10,0.25,0.35,0.50,0.75,0.85])

0.010    0.002697
0.025    0.002965
0.050    0.004050
0.100    0.004849
0.250    0.007389
0.350    0.008677
0.500    0.013067
0.750    0.137016
0.850    0.826105
Name: proba, dtype: float64

In [57]:
def risco(row):
    if row['proba']<=0.002697:
        return 1
    elif row['proba']<=0.002965:
        return 2
    elif row['proba']<=0.004050:
        return 3
    elif row['proba']<=0.007389:
        return 4
    elif row['proba']<=0.008677:
        return 5
    elif row['proba']<=0.013067:
        return 6
    elif row['proba']<=0.137016:
        return 7
    elif row['proba']<=0.826105:
        return 8
    else:
        return 9

df['nivel_risco'] = df.apply(risco, axis=1) 

In [58]:
df[['proba','CNPJ8','nivel_risco']].groupby(['nivel_risco']).count()

,proba,CNPJ8
nivel_risco,,
1,1050,1050
2,3010,3010
3,1968,1968
4,25266,25266
5,13308,13308
6,18384,18384
7,31151,31151
8,12552,12552
9,18827,18827


In [84]:
df[['proba','CNPJ8','ANO','nivel_risco']].to_csv('bndes_escorado.csv')

<a name="raroc"></a>

## 3.3) Calcular RAROC 
  
- ir para [Menu Principal](#navegacao)

In [ ]:
DATA RAROC;SET PD;
VALOR_OPERACOES   = 35000;
GARANTIA          = 0.38;
VALOR_FINAL       = VALOR_OPERACOES*((1+TAXA_JUROS)**PRAZO);
PRESTACAO_FORMULA = VALOR_OPERACOES*((((1+TAXA_JUROS)**PRAZO)*TAXA_JUROS)/(((1+TAXA_JUROS)**PRAZO)-1));
PRESTACAO_PRICE   = FINANCE('pmt',TAXA_JUROS,PRAZO,-VALOR_OPERACOES,0);
SELIC             =  0.00525383 ;
SELIC_PRAZO       = ((1+SELIC)**PRAZO)-1;
CUSTO_CAPTACAO    = VALOR_OPERACOES*(1+SELIC_PRAZO);
MONTANTE_NOMINAL  = PRAZO*PRESTACAO_PRICE;
CUSTOS_OPER       = 0.02*MONTANTE_NOMINAL;
PD                = PD*(1+0.27)**(PRAZO*0.27);
PERDA_ESPERADA    = PD*(1-GARANTIA)*VALOR_OPERACOES;
SPREAD            = MONTANTE_NOMINAL-CUSTO_CAPTACAO;
LUCRO_AJUSTADO    = SPREAD-CUSTOS_OPER-PERDA_ESPERADA;
VAR               = PERDA_ESPERADA+(1-GARANTIA)*(QPROV99*VALOR_OPERACOES);
RAROC             = LUCRO_AJUSTADO / VAR;
RUN;

In [92]:
%%time
#https://pbpython.com/amortization-model.html
import pandas as pd
rar = pd.read_csv("BNDESOPERACOES.csv", delimiter=";" , encoding='latin-1', decimal=',',
                  converters={'EMPRESA': str,'Valor_Contrato': np.float64,'Valor_Desembolso': np.float64})

ValueError: could not convert string to float: '745,000'

In [88]:
rar.head()

,EMPRESA,EMPRESA5,NOME,Cliente,CNPJ,RAZAO_SOCIAL2,UF,Municipio,Codigo_Municipio,Data_Contrato,...,Instrumento_Financeiro,Inovacao,Area_Operacional,Setor_CNAE,Setor_BNDES,Porte_Cliente,Natureza_Cliente,Instituicao_Credenciada,CNPJ_Instituicao,Situacao_Contrato
0,NaN,16040,jjsdiagnosticop,''J.J.S. DIAGNOSTICO POR IMAGEM LTDA.'',**.*16.040/0001-**,NaN,SP,LIMEIRA,3526902,09AUG2022,...,LINHA EMPRÉSTIMO PARA MICRO E PEQUENAS EMPRESAS,NÃO,AREA DE OPERACOES E CANAIS DIGITAIS,COMERCIO E SERVICOS,COMERCIO/SERVICOS,MÉDIA,PRIVADA,ITAU UNIBANCO S.A.,60.701.190.0001-04,ATIVA.
1,NaN,16040,jjsdiagnosticop,''J.J.S. DIAGNOSTICO POR IMAGEM LTDA.'',**.*16.040/0001-**,NaN,SP,LIMEIRA,3526902,11AUG2017,...,BK AQUISIÇÃO E COMERCIALIZAÇÃO,NÃO,AREA DE OPERACOES E CANAIS DIGITAIS,COMERCIO E SERVICOS,COMERCIO/SERVICOS,MÉDIA,PRIVADA,BANCO DE LAGE LANDEN BRASIL S.A.,05.040.481.0001-82,LIQUIDADA.
2,NaN,17525,lanzacialtda,''LANZA & CIA LTDA.'',**.*17.525/0001-**,NaN,SP,LEME,3526704,05APR2022,...,LINHA EMPRÉSTIMO PARA MICRO E PEQUENAS EMPRESAS,NÃO,AREA DE OPERACOES E CANAIS DIGITAIS,COMERCIO E SERVICOS,COMERCIO/SERVICOS,PEQUENA,PRIVADA,BANCO SANTANDER (BRASIL) S.A.,90.400.888.0001-42,ATIVA.
3,7290688.0,90688,metalizarevesti,''METALIZA REVESTIMENTOS INDUSTRIAIS LTDA,**.*90.688/0001-**,''METALIZA REVESTIMENTOS INDUSTRIAIS LTDA,SP,SOROCABA,3552205,18NOV2022,...,LINHA EMPRÉSTIMO PARA MICRO E PEQUENAS EMPRESAS,NÃO,AREA DE OPERACOES E CANAIS DIGITAIS,INDUSTRIA DE TRANSFORMAÇÃO,INDUSTRIA,MÉDIA,PRIVADA,ITAU UNIBANCO S.A.,60.701.190.0001-04,ATIVA.
4,3467919.0,67919,morenobarbieril,''MORENO & BARBIERI LTDA,**.*67.919/0001-**,''MORENO & BARBIERI LTDA,SP,CERQUEIRA CESAR,3511409,25JUL2018,...,BNDES GIRO,NÃO,AREA DE OPERACOES E CANAIS DIGITAIS,COMERCIO E SERVICOS,COMERCIO/SERVICOS,MICRO,PRIVADA,BANCO BRADESCO S.A.,60.746.948.0001-12,ATIVA.


In [89]:
rar.dtypes

EMPRESA                    float64
EMPRESA5                     int64
NOME                        object
Cliente                     object
CNPJ                        object
RAZAO_SOCIAL2               object
UF                          object
Municipio                   object
Codigo_Municipio             int64
Data_Contrato               object
Valor_Contrato              object
Valor_Desembolso            object
Fonte_Recurso               object
Custo_Financeiro            object
Juros                      float64
Prazo_Carencia               int64
Prazo_Amortizacao            int64
Modalidade_Apoio            object
Forma_Apoio                 object
Produto                     object
Instrumento_Financeiro      object
Inovacao                    object
Area_Operacional            object
Setor_CNAE                  object
Setor_BNDES                 object
Porte_Cliente               object
Natureza_Cliente            object
Instituicao_Credenciada     object
CNPJ_Instituicao    

In [72]:
%%time
df.loc[ (df['GARANTIA']==1) ,'GARANTIA'] = 0.35
df.loc[~(df['GARANTIA']==1) ,'GARANTIA'] = 0.00

df['VALOR_OPERACOES']   = df['VALOR_CONTRATO']
df['VALOR_FINAL']       = df['VALOR_OPERACOES']*((1+df['MEDIA_JUROS'])**df['PRAZO_AMORTIZACAO'])
df['PRESTACAO_FORMULA'] = df['VALOR_OPERACOES']*((((1+df['MEDIA_JUROS'])**df['PRAZO_AMORTIZACAO'])*df['MEDIA_JUROS']) /  
(((1+df['MEDIA_JUROS'])**df['PRAZO_AMORTIZACAO'])-1))

df['SELIC']             = 0.00525383
df['SELIC_PRAZO']       = (df['SELIC']*df['PRAZO_AMORTIZACAO'])-1
df['CUSTO_CAPTACAO']    = df['VALOR_OPERACOES']*((1-df['SELIC_PRAZO']))
df['MONTANTE_NOMINAL']  = df['PRAZO_AMORTIZACAO']*df['PRESTACAO_FORMULA']
df['CUSTOS_OPER']       = df['MONTANTE_NOMINAL']*0.02
df['PD']                = df['proba']*(1+0.27)**(df['PRAZO_AMORTIZACAO'])
df['PERDA_ESPERADA']    = df['proba']*(1-df['GARANTIA'])*df['VALOR_OPERACOES']
df['SPREAD']            = df['MONTANTE_NOMINAL']-df['CUSTO_CAPTACAO']
df['LUCRO_AJUSTADO']    = df['SPREAD']-df['CUSTOS_OPER']-df['PERDA_ESPERADA']
df['QPROV99']           = df['proba']+((df['proba']*(1-df['proba']))*1.95)
df['VAR']               = df['PERDA_ESPERADA']+((1-df['GARANTIA'])*(df['QPROV99']*df['VALOR_OPERACOES']))
df['RAROC']             = df['LUCRO_AJUSTADO']/df['VAR']

CPU times: total: 93.8 ms
Wall time: 84.8 ms


In [83]:
#df[['VALOR_OPERACOES','MEDIA_JUROS','PRAZO_AMORTIZACAO','PRESTACAO_FORMULA','TEMP']].head()
df.head()

,CNPJ8,ANO,EMPRESA_PUBLICA,GARANTIA,INDIRETA,INOVACAO,INSTRUMENTO,UF,INTERMEDIARIA,CUSTO,...,MONTANTE_NOMINAL,CUSTOS_OPER,PD,PERDA_ESPERADA,SPREAD,LUCRO_AJUSTADO,QPROV99,VAR,RAROC,TEMP
0,00000000,2002,0,0.0,0,1,OUTROS,IE,00000000,TJLP,...,NaN,NaN,2.286003e+06,86270257.627497,NaN,NaN,1.057090,184352500.83399,NaN,inf
1,00000000,2003,0,0.0,0,0,OUTROS,IE,92816560,TJLP,...,NaN,NaN,4.590564e+05,6333418.459567,NaN,NaN,1.065582,13695168.017453,NaN,inf
2,00000000,2009,1,0.0,0,0,OUTROS,RJ,,TAXAFIXA,...,5.015477e+18,1.003095e+17,2.588300e+00,2213032950.386658,5.015477e+18,4915167148177067008.0,1.063806,4773238161.398135,1029734319.130927,3.911719e+37
3,00000000,2010,1,0.0,0,0,OUTROS,RJ,,TAXAFIXA,...,6.052936e+19,1.210587e+18,6.792703e+01,2384720331.377283,6.052936e+19,59318775237632958464.0,1.063806,5143546592.021261,11532660232.853542,1.073921e+164
4,00000000,2012,1,0.0,0,0,OUTROS,IE,,TJLP,...,NaN,NaN,2.451524e+22,5711213875.007984,NaN,NaN,1.057884,12217512619.810976,NaN,inf


In [81]:
df['TEMP'] = ((1+df['MEDIA_JUROS'])**df['PRAZO_AMORTIZACAO'])